In [ ]:
# Prerequisitos

In [2]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine, inspect, text

In [3]:
USERNAME = 'postgres'
PASSWORD = '1978'
HOST = 'localhost'
DATABASE = 'classicmodels'

In [4]:
engine_1 = create_engine(f"postgresql://{USERNAME}:{PASSWORD}@{HOST}/{DATABASE}")


In [5]:
revision = inspect(engine_1)

In [6]:
tablas = revision.get_table_names()
tablas

['customers',
 'employees',
 'offices',
 'orderdetails',
 'orders',
 'payments',
 'productlines',
 'products',
 'top_10_clientes_2005',
 'top_10_productos_2005']

In [7]:
#Genera una función llamada leer_tabla(tabla, engine) y utilízala para leer tablas completas desde la base de datos en dataframes independientes.
# Entendi el engine como el despliegue de todas las entradas necesarias para obtener conexion correcta.
def leer_tabla(tabla,USERNAME = 'postgres', PASSWORD = '1978' , HOST = 'localhost', DATABASE = 'classicmodels'):
        db_url_1 = (f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:5432/{DATABASE}")
        engine_1 = create_engine(db_url_1)
        revision = inspect(engine_1)
        tablas = revision.get_table_names()
        query_order = f"SELECT * FROM {tabla};"
        df_tabla = pd.read_sql(query_order, engine_1)
        return df_tabla
        df_tabla



In [8]:
df_customers = leer_tabla('customers')
df_employees = leer_tabla('employees')
df_orderdetails = leer_tabla('orderdetails ')
df_orders = leer_tabla('orders')
df_products = leer_tabla('products ')

In [9]:
#Realiza el cruce entre los DataFrames, asegurándote de utilizar correctamente el parámetro validate para asegurar la integridad referencial.
df_merged = df_orderdetails.merge(df_orders,on = 'orderNumber',how ='left', validate = 'many_to_one')

In [10]:
df_merged1 = df_merged.merge(df_customers,on = 'customerNumber',how ='left', validate = 'many_to_one')

In [11]:
df_merged2 = df_merged1.merge(df_products,on = 'productCode',how ='left', validate = 'many_to_one')

In [12]:
df_merged3 = df_merged2.merge(df_employees,left_on = 'customerNumber',right_on = 'employeeNumber'  ,how ='left', validate = 'many_to_one')

In [13]:
df_merged3.head(1)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,...,buyPrice,MSRP,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,10100,S18_1749,30,136.0,3,2003-01-06,2003-01-13,2003-01-10,Shipped,None,...,86.7,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Agrega las siguientes columnas, considerando su nombre y la fórmula asociada
df_merged3['venta'] = df_merged3['quantityOrdered'] * df_merged3['priceEach']

In [15]:
df_merged3['costo'] = df_merged3['quantityOrdered'] * df_merged3['buyPrice']

In [16]:
df_merged3['ganancia'] = df_merged3['venta'] * df_merged3['costo']

In [17]:
#¿Cuál fue el total de ventas por línea de productos? Incluye una fila de totales.

df_base_reporte = df_merged3.pivot_table(
                                  index = "productLine"
                                  ,values = ("venta","costo", "ganancia")
                                  , aggfunc= "sum" 
                                 , margins = True
                                            )

df_base_reporte




,costo,ganancia,venta
productLine,,,
Classic Cars,2327710.29,1.088378e+10,3853922.49
Motorcycles,652170.82,2.535885e+09,1121426.12
Planes,588676.83,1.963637e+09,954637.54
Ships,402708.87,1.237745e+09,663998.34
Trains,123191.90,3.337633e+08,188532.92
Trucks and Buses,623560.35,2.431270e+09,1024113.57
Vintage Cars,1060291.30,3.644441e+09,1797559.63
All,5778310.36,2.303052e+10,9604190.61


In [ ]:
#¿Cuántos clientes distintos hicieron compras?

In [18]:
df_mergfed_aux = df_merged3[df_merged3['quantityOrdered'] > 0]

In [19]:
b=np.array(df_mergfed_aux['customerNumber'].value_counts())

In [20]:
print(b.size)

98


In [21]:
df_mergfed_aux['customerNumber'].unique()

array([363, 128, 181, 121, 141, 145, 278, 131, 385, 486, 187, 129, 144,
       124, 172, 424, 381, 148, 216, 382, 114, 353, 350, 103, 112, 458,
       151, 324, 198, 447, 323, 250, 242, 496, 282, 161, 334, 320, 205,
       379, 276, 487, 311, 333, 219, 186, 473, 321, 347, 227, 452, 462,
       448, 233, 175, 386, 344, 171, 167, 339, 484, 489, 211, 259, 471,
       146, 319, 455, 475, 357, 202, 495, 177, 406, 256, 166, 189, 314,
       239, 298, 173, 240, 328, 412, 260, 209, 456, 450, 201, 398, 362,
       157, 119, 204, 249, 299, 286, 415], dtype=int64)

In [22]:
#6. ¿Existen clientes que aún no han hecho ninguna compra? ¿Cuántos son?


merged_aux2 = df_merged3.merge(df_customers,'outer',on = 'customerNumber', indicator=True).query('_merge == "right_only"')

merged_aux2.head()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,...,phone_y,addressLine1_y,addressLine2_y,city_y,state_y,postalCode_y,country_y,salesRepEmployeeNumber_y,creditLimit_y,_merge
2996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(26) 642-7555,ul. Filtrowa 68,None,Warszawa,None,01-012,Poland,NaN,0.0,right_only
2997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2035557845,149 Spinnaker Dr.,Suite 101,New Haven,CT,97823,USA,1286.0,0.0,right_only
2998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(1) 356-5555,Estrada da saúde n. 58,None,Lisboa,None,1756,Portugal,NaN,0.0,right_only
2999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,+612 9411 1555,Suntec Tower Three,8 Temasek,Singapore,None,038988,Singapore,NaN,0.0,right_only
3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0372-555188,Taucherstraße 10,None,Cunewalde,None,01307,Germany,NaN,0.0,right_only


In [23]:
c= merged_aux2['customerNumber'].value_counts()

In [24]:
print(c.size)

24


In [26]:
merged_aux2['customerNumber'].unique()

array([125, 168, 169, 206, 223, 237, 247, 273, 293, 303, 307, 335, 348,
       356, 361, 369, 376, 409, 443, 459, 465, 477, 480, 481], dtype=int64)

In [27]:
#Cuáles fueron los 10 clientes que reportan mayores ventas brutas en dinero durante el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada
#top_10_clientes_2005, en la que se especifique el nombre del cliente y su correspondiente venta, costo y ganancia.


df_merged_aux3 = df_merged3[pd.to_datetime(df_merged3['orderDate']).dt.year == 2005]

In [28]:


df_base_reporte2 = df_merged_aux3.pivot_table(
                                  index = "customerName"
                                  ,values = ("venta","costo", "ganancia")
                                  , aggfunc= "sum" 
                                 , margins = False
                                           ).sort_values("venta",ascending=False).head(10)
                                              
                                            
df_base_reporte2


,costo,ganancia,venta
customerName,,,
Euro+ Shopping Channel,169989.97,7.395476e+08,290018.52
Mini Gifts Distributors Ltd.,115084.72,4.563321e+08,192481.73
La Rochelle Gifts,55527.04,2.275532e+08,91147.11
The Sharp Gifts Warehouse,50843.02,2.888998e+08,83984.89
"Down Under Souveniers, Inc",46389.52,2.266632e+08,75020.13
"Anna's Decorations, Ltd",35414.90,1.344142e+08,56932.30
Salzburg Collectables,33536.26,1.643361e+08,52420.07
Gifts4AllAges.com,33221.25,1.342891e+08,50806.85
Corporate Gift Ideas Co.,28561.31,1.188139e+08,46781.66


In [29]:
df_base_reporte2.to_sql('top_10_clientes_2005', engine_1, if_exists='replace')


10

In [87]:
#¿Cuál fue el top 10 de artículos más vendidos durante el año 2005 (considerando cantidad neta)? Genera un DataFrame y guárdalo en una tabla de Postgre llamada
#top_10_productos_2005, en la que se especifique el nombre del producto y su correspondiente venta, costo y ganancia.
# Entendi cantidad neta como cantidad ordenada.

In [30]:
df_base_reporte3 = df_merged_aux3.pivot_table(
                                  index = "productName"
                                  ,values = ("quantityOrdered","venta","costo", "ganancia" )
                                  , aggfunc={"quantityOrdered": "count", "venta" : "sum",   "costo": "sum", "ganancia": "sum" }
                                   , margins = False
                                           ).sort_values("quantityOrdered",ascending=False).head(10).drop(columns=["quantityOrdered"])
                                              
                                            
df_base_reporte3


,costo,ganancia,venta
productName,,,
1992 Ferrari 360 Spider red,27031.30,1.756734e+08,52978.28
2002 Suzuki XREO,15308.37,9.053502e+07,30434.09
1960 BSA Gold Star DBD34,9031.44,2.787512e+07,16049.47
1936 Harley Davidson El Knucklehead,4967.15,1.040867e+07,11195.21
1997 BMW F650 ST,17198.44,6.559288e+07,22469.91
1997 BMW R 1100 S,16493.06,8.063628e+07,28747.69
2003 Harley-Davidson Eagle Drag Bike,19023.18,1.437572e+08,34856.12
18th Century Vintage Horse Carriage,8503.60,2.323529e+07,13110.81
1982 Camaro Z28,7863.57,2.493202e+07,15037.77


In [31]:
df_base_reporte3.to_sql('top_10_productos_2005', engine_1, if_exists='replace')

10

In [ ]:
# Una función que permita filtrar un DataFrame por fechas, indicando dataframe, columna para filtrar, fecha inicio y fecha fin. La función debe retornar un DataFrame.

In [101]:
# la Funcion creada en archivo funciones.py

def filtra_por_fecha(df, columna, fecha_inicio, fecha_fin):
    """
    Genera un reporte pivoteado

    Argumentos:
    dataframe (df): DataFrame de origen.
    Columna: columna a filtrar.    
    fecha_inicio: fecha inicio de filtro.
    fecha_fin: fecha inicio de filtro.

    Retorno
    DataFrame: DataFrame pivoteado.
    """
    
    
    
    fecha_inicio = pd.to_datetime(fecha_inicio, format="%Y-%m-%d")
    fecha_fin =  pd.to_datetime(fecha_fin, format="%Y-%m-%d")
    mascara = (pd.to_datetime(df[columna]).dt.date >= fecha_inicio.date())  & (pd.to_datetime(df[columna]).dt.date <= fecha_fin.date())
    return df[mascara]





In [118]:
# Importar todas las función creadas en las tres preguntas siguientes.
import funciones as fc

In [103]:
filtra_por_fecha(df_merged3,'orderDate',"2003-01-06", "2003-01-06")

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,orderDate,requiredDate,shippedDate,status,comments,...,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,venta,costo,ganancia
0,10100,S18_1749,30,136.00,3,2003-01-06,2003-01-13,2003-01-10,Shipped,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4080.00,2601.00,1.061208e+07
1,10100,S18_2248,50,55.09,2,2003-01-06,2003-01-13,2003-01-10,Shipped,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2754.50,1665.00,4.586242e+06
2,10100,S18_4409,22,75.46,4,2003-01-06,2003-01-13,2003-01-10,Shipped,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1660.12,951.72,1.579969e+06
3,10100,S24_3969,49,35.29,1,2003-01-06,2003-01-13,2003-01-10,Shipped,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1729.21,1065.75,1.842906e+06


In [104]:
#Una función que permita generar reportes dependiendo de parámetros de entrada como dataframe, filas, columnas, valores y medida (funcion_agrupadora). Utilizar
#fill_value = 0. Esta función debe retornar un DataFrame pivotado.
# la Funcion creada en archivo funciones.py

def generar_reporte(df, filas, columnas, valores, funcion_agrupadora):
    """
    Genera un reporte pivoteado

    Argumentos:
    dataframe (df): DataFrame de origen.
    filas: Nombre de la columna a utilizar o lista de columnas.
    columnas:Nombre de la columna a utilizar o lista de columnas.
    valores: Nombre de la columna a utilizar o lista de columnas.
    funcion_agrupadora : Función agrupadora a utiliozar ()'sum', 'mean', 'count', etc).

    Retorno
    DataFrame: DataFrame pivoteado.
    """
  
    pivote = pd.pivot_table(df, values=valores,
                            index=filas, columns=columnas, 
                            aggfunc=funcion_agrupadora, 
                            fill_value=0)
    
    return pivote




In [107]:
generar_reporte(df_merged3,['productName','orderDate'], 'customerName', 'costo', 'sum')

customerName                                    AV Stores, Co.  Alpha Cognac  \
productName                         orderDate                                  
18th Century Vintage Horse Carriage 2003-01-29             0.0          0.00   
                                    2003-03-25             0.0          0.00   
                                    2003-05-28             0.0          0.00   
                                    2003-07-16             0.0          0.00   
                                    2003-09-19             0.0          0.00   
...                                                        ...           ...   
The USS Constitution Ship           2004-12-16             0.0          0.00   
                                    2005-01-31             0.0          0.00   
                                    2005-03-01             0.0          0.00   
                                    2005-03-28             0.0        747.34   
                                    2005-05-06             0.0          0.00   

customerName                                    Amica Models & Co.  \
productName                         orderDate                        
18th Century Vintage Horse Carriage 2003-01-29                 0.0   
                                    2003-03-25                 0.0   
                                    2003-05-28                 0.0   
                                    2003-07-16                 0.0   
                                    2003-09-19                 0.0   
...                                                            ...   
The USS Constitution Ship           2004-12-16                 0.0   
                                    2005-01-31                 0.0   
                                    2005-03-01                 0.0   
                                    2005-03-28                 0.0   
                                    2005-05-06                 0.0   

customerName                                    Anna's Decorations, Ltd  \
productName                         orderDate                             
18th Century Vintage Horse Carriage 2003-01-29                      0.0   
                                    2003-03-25                      0.0   
                                    2003-05-28                      0.0   
                                    2003-07-16                      0.0   
                                    2003-09-19                      0.0   
...                                                                 ...   
The USS Constitution Ship           2004-12-16                      0.0   
                                    2005-01-31                      0.0   
                                    2005-03-01                      0.0   
                                    2005-03-28                      0.0   
                                    2005-05-06                      0.0   

customerName                                    Atelier graphique  \
productName                         orderDate                       
18th Century Vintage Horse Carriage 2003-01-29                0.0   
                                    2003-03-25                0.0   
                                    2003-05-28                0.0   
                                    2003-07-16                0.0   
                                    2003-09-19                0.0   
...                                                           ...   
The USS Constitution Ship           2004-12-16                0.0   
                                    2005-01-31                0.0   
                                    2005-03-01                0.0   
                                    2005-03-28                0.0   
                                    2005-05-06                0.0   

customerName                                    Australian Collectables, Ltd  \
productName                         orderDate                                  
18th Century V

In [116]:
engine =engine_1

In [117]:
#Una función que permita escribir en la base de datos a través del guardado de un DataFrame dependiendo de parámetros de entrada como DataFrame, nombre de la
#tabla, engine y comportamiento en caso de que exista la tabla (if_exists).

# la Funcion creada en archivo funciones.py
def escribir_en_base_de_datos(df, nombre_tabla, engine, if_exists='fail'):
    """
    Escribe un DataFrame en una base de datos.

    Argumentos:
    dataframe (df): DataFrame de origen.
    nombre_tabla: El nombre de la tabla que se guardara en la base datos.
    engine: El motor de la base de datos.
    if_exists puede ser 'fail', 'replace', 'append'. Por defecto, 'fail'.

    Returns:
    None
    """
    # Escribir el DataFrame en la base de datos
    df.to_sql(nombre_tabla, engine, if_exists=if_exists, index=False)


In [119]:
escribir_en_base_de_datos(df_merged3, 'Guardar_df_bd_2', engine)